In [1]:
from dotenv import load_dotenv
load_dotenv(".env.local")

True

## .docx to text

### AzureAI Document Intelligence

In [78]:
from langchain_community.document_loaders.doc_intelligence import (
    AzureAIDocumentIntelligenceLoader,
)
import os
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
def load_docs_adi(filepath: str):
    adi = AzureAIDocumentIntelligenceLoader(
        "https://eastus.api.cognitive.microsoft.com",
        AZURE_DOCUMENT_INTELLIGENCE_API_KEY,
        file_path=filepath,
        api_model="prebuilt-read",
    )
    docs = adi.load()
    return docs

### PyPDF

In [85]:
from langchain_community.document_loaders import PyPDFLoader

def load_docs_pypdf(file_path: str):
    pypdf_loader = PyPDFLoader(
        file_path = file_path,
        password = None,
        extract_images = True,
        # headers = None
        # extraction_mode = "plain",
        # extraction_kwargs = None,
    )
    docs = pypdf_loader.load()
    return docs

### Zerox

In [2]:
from langchain.schema import Document
def load_docs_zerox(path:str):
    with open(path, "r") as f:
        data = f.read()
    return [Document(page_content=data)]

In [4]:
import os
current_dir = os.getcwd()
#file_path = os.path.join(current_dir, "RA Duty Manual - Final Draft 2023-2024.pdf")
file_path = os.path.join(current_dir, "./zerox/output.md")

docs = load_docs_zerox(file_path)

## Pinecone

In [7]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone.grpc import GRPCVector
from pinecone.grpc.index_grpc import UpsertResponse

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_HOST = os.getenv("PINECONE_HOST")
PINECONE_NAMESPACE = "umd-call-center"
pc = Pinecone(api_key=PINECONE_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
PC_INDEX_NAME = "knowledge"
pc_index = pc.Index(PC_INDEX_NAME, host=PINECONE_HOST)

In [ ]:
from typing import List
def query_pc(vector: List[float]):
    query_result = pc_index.query(
        vector=vector,
        namespace=PINECONE_NAMESPACE,
        top_k=10,
        #filter={"knowledge_uuid": {"$in": knowledge_uuids}},
        include_metadata=True,
        timeout=1,
    )
    return query_result


## Create Local Embeddings

In [134]:
from langchain_openai import OpenAIEmbeddings
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

embedder = OpenAIEmbeddings(
    model="text-embedding-3-large", api_key=OPENAI_API_KEY
)

In [47]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_by_semantics(content:str):
    semantic_splitter = SemanticChunker(
        embedder,
        add_start_index=True,
        sentence_split_regex=r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)(?=\s)",
    )
    splits = semantic_splitter.split_documents([Document(page_content=content)])
    start_index = 0
    for i, split in enumerate(splits):
        last_index = start_index + len(content) - 1
        split.metadata["start_index"] = start_index
        start_index = last_index + 1
    return splits

def split_by_character(content:str, chunk_size:int=800, overlap:int=400):
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=overlap, add_start_index=True
    )
    splits = recursive_splitter.split_documents([Document(page_content=content)])
    return splits

## Generate Test Dataset

## Measuring

### Retrieval relevence
Are the documents that were retrieved relevent to the query

### Answer Faithfulness
Is the answer grounded in the documents?
- This can measure hallucinations

### Answer Correctness
Is the answer consistent with a reference answer?

### Code

#### Gen dataset giskard

In [55]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
splits = []
chunk_size = 800
chunk_overlap = 0
for doc in docs:
    splits.extend(split_by_character(doc.page_content, chunk_size=chunk_size, overlap=overlap))

In [56]:
vector_store = InMemoryVectorStore(OpenAIEmbeddings(model="text-embedding-3-large"))
vector_store.add_documents(splits)


['0b8fe8af-e795-482a-8186-f96ace3bc29b',
 'a021b58f-269b-448f-a9f4-2d1e228c3f83',
 'a9098cc2-c426-48c5-a347-362ef40e386f',
 '49fa7c4b-5ec2-4eea-824a-7bbe4493080f',
 'c5a9e988-1f48-411f-8302-5ad8c183c14a',
 '7bdbd29c-da1d-4161-a57e-ba483c5975fe',
 '2d8fddb2-ad13-4ded-80a8-d53bf9e79b10',
 '9eb306e3-62ff-4baa-ab6d-78fb0ff5a46c',
 '35891cd3-a1ec-4f1a-9262-0872b7b2204b',
 '8974aa7e-8070-4efe-8922-38349010e279',
 '04aa3f91-b8a8-4f51-8e4d-83e86f5f69d8',
 '2d7e7aac-410e-43f6-bdb2-a02f5f38a620',
 'fdd429a0-f69c-4abc-8a1b-213fe5f09346',
 'a426fc5f-f818-47dc-afc5-6a1a8c9f64b3',
 'd2d829f7-dbc3-43b4-bec6-3eb6214bd0d0',
 '216ebf1a-97cc-4013-9405-7f625e6af91d',
 '741d2fd3-1723-4700-a857-f049888084a3',
 '46d9db90-6e85-4aed-9afc-9d8690122e0f',
 '4ee49b6a-2eaf-4ca0-9349-e93033b1592e',
 '3b09d263-6d5d-4d95-8246-6c522c495698',
 '75abdffa-738e-4e21-b7d6-ed0f57f82f8f',
 'f77c9528-7abd-4280-8546-45243e38a5da',
 '2f22285e-11c7-457d-8f37-2096c9145baf',
 'be39b362-d315-4e40-b90b-3ef0b7d3eb6e',
 '9d966b5c-4f6e-

In [215]:
vector_store.dump("vector_store")

In [7]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
vector_store = InMemoryVectorStore.load("vector_store", OpenAIEmbeddings(model="text-embedding-3-large"))

In [12]:
import pandas as pd
from giskard.rag import KnowledgeBase
knowledge_base_df = pd.DataFrame([i.page_content for i in splits], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)

In [213]:
from giskard.rag import RAGReport
report = RAGReport.load("reports_testset250/p1-gpt_4o_mini-splits_character-max_marginal_relevance-top5-zerox-tester_gpt-4o-mini")
knowledge_base = report._knowledge_base

In [221]:
import giskard
from giskard.llm.client.openai import OpenAIClient

giskard.llm.set_llm_api("openai")
oc = OpenAIClient(model="gpt-4o-mini")

In [14]:
from typing import Any, Optional, Sequence, Union
from giskard.llm.client import LLMClient, ChatMessage
import openai
from openai.types.chat.completion_create_params import ResponseFormatJSONObject, ResponseFormatText
import giskard
class O1Client(LLMClient):
    def __init__(self, model_openai = "o1-preview") -> None:
        super().__init__()
        self.model_openai = model_openai


    def complete(self, messages: Sequence[ChatMessage], temperature: float = 1, max_tokens: Union[Optional[int], Any] = None, caller_id: str | None = None, seed: int | None = None, format=None) -> ChatMessage:
        messages_openai = []
        for message in messages:
            role = message.role
            if message.role != "user" or message.role != "assistant":
                role = "user"
            messages_openai.append({"role": role, "content": message.content})
            
        if format is not None and "json" in format:
            rformat: ResponseFormatJSONObject = {"type": "json_object"}
        else:
            rformat: ResponseFormatText = {"type": "text"}

        if max_tokens is None:
            response = openai.chat.completions.create(messages=messages_openai, model=self.model_openai, temperature=1,seed=seed, response_format=rformat)
        else:
            response = openai.chat.completions.create(messages=messages_openai, model=self.model_openai, temperature=1, max_tokens=max_tokens, seed=seed, response_format=rformat)
            
        cm = ChatMessage(role=response.choices[0].message.role, content=response.choices[0].message.content)
        return cm
    
o1_client= O1Client(model_openai="o1-mini")
print(o1_client.model_openai)
giskard.llm.set_default_client(o1_client)


o1-mini


In [15]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=55,
    language='en',  
    agent_description="A support agent for the University of Maryland's call center.", 
)

2024-10-29 13:18:30,547 pid:14031 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-10-29 13:18:47,080 pid:14031 MainThread giskard.rag  INFO     Found 11 topics in the knowledge base.


Generating questions:   0%|          | 0/55 [00:00<?, ?it/s]

SELF  o1-mini


Generating questions:   2%|▏         | 1/55 [00:06<06:04,  6.75s/it]

SELF  o1-mini


...

Generating questions:  96%|█████████▋| 53/55 [14:32<00:54, 27.03s/it]

SELF  o1-mini
SELF  o1-mini


Generating questions:  98%|█████████▊| 54/55 [14:59<00:27, 27.10s/it]

SELF  o1-mini
SELF  o1-mini


Generating questions: 100%|██████████| 55/55 [15:15<00:00, 16.64s/it]


In [16]:
testset.to_pandas().to_excel("testset55-o1mini.xlsx")

In [200]:
testset.save("testset45-o1preview.jsonl")

In [40]:
from giskard.rag import QATestset
import json
import ast

def safe_literal_eval(cell):
    try:
        return ast.literal_eval(cell) if isinstance(cell, str) else cell
    except (ValueError, SyntaxError):
        print(f"Warning: Could not evaluate literal in cell: {cell}")
        return cell

df = pd.read_excel("./testset100-o1.xlsx")
# Apply the functions to the specific columns after loading
df['metadata'] = df['metadata'].apply(safe_literal_eval)
df['conversation_history'] = df['conversation_history'].apply(safe_literal_eval)

testset = QATestset.from_pandas(df)

In [140]:
from giskard.rag import QATestset
testset = QATestset.load("testset250.jsonl")

#### Evaluate gkiskard

In [61]:
from typing import Literal, Sequence, List
import giskard.llm
from giskard.llm.client.base import ChatMessage
from giskard.rag import evaluate
import openai

prompt_type: Literal["api", "p1", "p2"] = "p2"
retrieval_type: Literal["similarity_search", "max_marginal_relevance"] = "similarity_search"
retrieval_meta: Literal["top3", "top5", "top10"] = "top5"

def get_answer_fn(question: str, history: Optional[List[Any]]=None) -> str:
    """A function representing your RAG agent."""
    # Format appropriately the history for your RAG agent
    messages = history if history else []
    if type(messages) == str:
        raise ValueError("History should be a list of ChatMessage objects, not string.")
    messages.append({"role": "user", "content": question})


    if retrieval_type=="similarity_search":
        similarity_search_with_score = vector_store.similarity_search_with_score(question, k=5)
        sources_str=""
        for i, doc_score in enumerate(similarity_search_with_score):
            doc, score = doc_score
            sources_str += f"Source {i}. Relevency Score: {score}:\n"+ doc.page_content + "\n\n"
    elif retrieval_type=="max_marginal_relevance":
        docs = vector_store.max_marginal_relevance_search(question, k=5)
        sources_str = ""
        for i, doc in enumerate(docs):
            sources_str += f"Source {i}:\n"+ doc.page_content + "\n\n"

    if prompt_type == "api":
        system_message = "Search results found the following information that might be relevent:\n\n" 

        messages.append({"role": "system", "content": system_message + sources_str})

        reminder_message = "Remember, you are on a phone call. Your response to the caller should be accurate and concise. Do not monologue. Here is the caller's message:"
        messages.append({"role": "system", "content": reminder_message})
    elif prompt_type == "p1":
        system_message = "Search results found the following information that might be relevent:\n\n" 
        messages.append({"role": "system", "content": system_message + sources_str})

        instructions = "You are helpful support agent who answers phone calls.\n Search results will be given to you to help you answer questions. Only use those results to answer questions. If a topic comes up that you don't know, do not answer. You are to concisely answer their question, instead of quoting the information.Never insert additional information. If something is unclear, ask for clarification."
        messages.insert(0, {"role": "system", "content": instructions})
        messages.append({"role": "system", "content": instructions})
    elif prompt_type == "p2":
        system_message = "Search results found the following information that might be relevent:\n\n" 
        messages.append({"role": "system", "content": system_message + sources_str})

        instructions = "Remember, you are helpful support agent who answers phone calls.\n Search results will be given to you to help you answer questions. Only use those results to answer questions. If a topic comes up that you don't know, do not answer. You are to concisely answer their question. If something is ambigious, ask for clarification."
        messages.insert(0, {"role": "system", "content": instructions})
        messages.append({"role": "system", "content": instructions})
    chatcompletion = openai.chat.completions.create(messages=messages, model="gpt-4o-mini")
    answer = chatcompletion.choices[0].message.content
    return answer


#ragas_context_recall = RagasMetric(name="RAGAS Context Recall", metric=context_recall, requires_context=True)

from giskard.llm.client import LLMClient
from openai.types.chat.completion_create_params import ResponseFormatJSONObject, ResponseFormatText
class TesterLLM(LLMClient):
    def complete(self, messages: Sequence[ChatMessage], temperature: float = 1, max_tokens: int | None = None, caller_id: str | None = None, seed: int | None = None, format=None) -> ChatMessage:
        messages_openai = [{"role": message.role, "content": message.content} for message in messages]
        if format is not None and "json" in format:
            rformat: ResponseFormatJSONObject = {"type": "json_object"}
        else:
            rformat: ResponseFormatText = {"type": "text"}
        response = openai.chat.completions.create(messages=messages_openai, model="gpt-4o-mini", temperature=temperature, max_tokens=max_tokens, seed=seed, response_format=rformat)
        
        cm = ChatMessage(role=response.choices[0].message.role, content=response.choices[0].message.content)
        return cm
giskard.llm.set_default_client(TesterLLM())
# giskard.llm.set_default_client(O1Client(model_openai="o1-mini"))
test_llm: Literal[None, "gpt-4o-mini", "o1-mini"] = "gpt-4o-mini"
llm_client= TesterLLM()

#test_llm: Literal[None, "gpt-4o-mini", "o1-mini"] = "o1-mini"
#llm_client= O1Client(model_openai="o1-mini")
report = evaluate(get_answer_fn, testset=testset, knowledge_base=knowledge_base, llm_client=TesterLLM())

CorrectnessMetric evaluation: 100%|██████████| 100/100 [01:21<00:00,  1.22it/s]


In [52]:
test_llm: Literal[None, "gpt-4o-mini", "o1-mini"] = "o1-mini"

In [ ]:
#display(report)  # if you are working in a notebook

# or save the report as an HTML file
#report.to_html("rag_eval_report.html")

In [59]:
chunk_overlap = 0

In [62]:
report.save(f"reports_testset100-o1/{prompt_type}-gpt_4o_mini-splits_character_{chunk_size}_{chunk_overlap}-{retrieval_type}-{retrieval_meta}-zerox-tester_{test_llm}")

In [ ]:
from giskard.rag import RAGReport
loaded_report = RAGReport.load("reports/...")

In [65]:
report._knowledge_base._to_

In [63]:
report.failures

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
9c63fed4-b956-4aa0-9501-741321f4f034,What happens if a student does not work throug...,"If a student does not work through Labor Day, ...",Document 147: summer benefits\nSummer housing ...,[],"{'question_type': 'simple', 'seed_document_id'...",If a student living in summer housing does not...,False,The agent provided a general response about im...
f29be739-0393-44a5-a519-7912dba4c9e1,What should a staff member do if they are goin...,They are expected to immediately call the Serv...,Document 45: If a staff member is unable to re...,[],"{'question_type': 'simple', 'seed_document_id'...",If a staff member knows they will be late for ...,False,The agent provided a general response about no...
fb530e2e-e211-41f1-9c11-3e5bc8078666,In order to be considered for a promotion to t...,An employee must work a minimum of 120 total h...,Document 135: a. Possess a satisfactory perfor...,[],"{'question_type': 'complex', 'seed_document_id...",The minimum total number of hours an employee ...,False,The agent did not provide the specific minimum...
c9e0790a-3be2-4adf-b9bd-7de3f0c01105,Could you specify the individuals who hold the...,"Only the Service Center Assistant Manager, Man...","Document 162: All keys, swipes and key rings i...",[],"{'question_type': 'complex', 'seed_document_id...",The authority to approve the signing out of ke...,False,The agent provided a broader list of roles tha...
94e282d7-352b-42f9-b78e-762e24a61c1d,What steps should a student take regarding the...,"If you have a student parking permit, remove i...",Document 55: If you have a student parking per...,[],"{'question_type': 'distracting element', 'seed...",If a student has received a written warning fo...,False,The agent provided a detailed response about g...
e049d396-2658-4b2f-8722-e42697cdc731,In the event that a substitute fails to report...,The original (scheduled/assigned) staff member...,Document 48: If the substitute does not report...,[],"{'question_type': 'distracting element', 'seed...",When a substitute fails to report for their sc...,False,The agent provided a detailed response about t...
14060525-e7cd-4fa3-b239-501a78dc994c,"Hi, I'm a staff member at the university and I...",The driver should take the vehicle out to Moto...,Document 171: Vehicles should never be returne...,[],"{'question_type': 'situational', 'seed_documen...",It's important to ensure that university vehic...,False,The agent provided general refueling procedure...
42d9bf93-4e26-4fef-b301-c31e7cd69bd8,"Hi there, I’m currently dealing with an unexpe...",If a staff member is not able to safely travel...,Document 63: In the event the University has a...,[],"{'question_type': 'situational', 'seed_documen...",If you are a staff member unable to safely tra...,False,The agent provided a detailed response about g...
ad2c7d12-7bb1-4710-9efc-16f83d2bbbeb,What is the main purpose of the Service Center...,The Service Center serves as the main communic...,Document 6: The Department of Residential Faci...,[],"{'question_type': 'double', 'original_question...",The main purpose of the Service Center is to p...,False,The agent provided a general description of th...


## Misc 

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone.grpc import GRPCVector
from pinecone.grpc.index_grpc import UpsertResponse

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_HOST = os.getenv("PINECONE_HOST")
PINECONE_NAMESPACE = "umd-call-center"
pc = Pinecone(api_key=PINECONE_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
PC_INDEX_NAME = "knowledge"
pc_index = pc.Index(PC_INDEX_NAME, host=PINECONE_HOST)

In [ ]:
from typing import List
def query_pc(vector: List[float]):
    query_result = pc_index.query(
        vector=vector,
        namespace=PINECONE_NAMESPACE,
        top_k=10,
        #filter={"knowledge_uuid": {"$in": knowledge_uuids}},
        include_metadata=True,
        timeout=1,
    )
    return query_result


In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    # Compute dot product
    dot_product = np.dot(vec1, vec2)
    # Compute magnitudes
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    # Calculate cosine similarity
    if magnitude_vec1 == 0 or magnitude_vec2 == 0:
        return 0  # Handle the case of zero magnitude
    return dot_product / (magnitude_vec1 * magnitude_vec2)
